In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold
import numpy as np

In [7]:
movies = pd.read_csv("../data/movies.csv")

In [9]:
mean_rating = movies['averageRating'].mean()

In [10]:
mean_rating

6.063703522333765

In [12]:
movies['error'] = mean_rating - movies['averageRating']

In [24]:
get_rmse

<function __main__.get_rmse()>

In [30]:
def rmse():
    out = 0
    for x in movies['error']:
        out += x**2

    return (out/movies['error'].size)**(0.5)

In [31]:
rmse()

1.2180205652684748